In [1]:
# Work around for using Autosklearn in Colab
!pip install git+https://github.com/Frankothe196/auto-sklearn.git@python3.10-added-compatibility;

  Cloning https://github.com/Frankothe196/auto-sklearn.git (to revision python3.10-added-compatibility) to /tmp/pip-req-build-dyvunogr
  Running command git clone --filter=blob:none --quiet https://github.com/Frankothe196/auto-sklearn.git /tmp/pip-req-build-dyvunogr
  Running command git checkout -b python3.10-added-compatibility --track origin/python3.10-added-compatibility
  Switched to a new branch 'python3.10-added-compatibility'
  Branch 'python3.10-added-compatibility' set up to track remote branch 'python3.10-added-compatibility' from 'origin'.
  Resolved https://github.com/Frankothe196/auto-sklearn.git to commit 80a575760e99945fa31970b1479edeb759bc645a
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 10.4 MB/s eta 0:00:00
  Preparing met

In [2]:
import sklearn.model_selection
from sklearn.datasets import fetch_openml
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

X, y = fetch_openml(data_id=40691, as_frame=True, return_X_y=True)
enc = OneHotEncoder(handle_unknown='ignore')
X = enc.fit_transform(X)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)

clf = RandomForestClassifier(random_state=41)
clf = clf.fit(X_train, y_train)
y_hat = clf.predict(X_test)
print("RF Accuracy", sklearn.metrics.accuracy_score(y_test, y_hat))

# Redo using pure data, instead of one-hot encoding -- One-Hot encoding is meant
# for use on categorical data. However, the values presented as 'observations'
# meant to serve as predictors for the 'quality' of the wine are continous real
# values. As such, preforming one-hot encoding is not the way to use it.
from autosklearn.classification import AutoSklearnClassifier
X, y = fetch_openml(data_id=40691, as_frame=True, return_X_y=True)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, random_state=42)
# After checking training and testing data, it was found that while the
# training data accuracy was good, the test accuracy was bad. So to avoid
# overfitting, trying out a different resampling strategy! As such,
# testing cv strategy with normal 10 folds.
# In addition; tried running with greater budget, slightly better, but not by a lot.

automl = AutoSklearnClassifier(time_left_for_this_task=300,resampling_strategy='cv',resampling_strategy_arguments={"folds": 10})
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)
print("AutoML Accuracy On Test", sklearn.metrics.accuracy_score(y_test, y_hat))

automl.refit(X_train,y_train)
y_h = automl.predict(X_train)
print("AutoML Accuracy on Training", sklearn.metrics.accuracy_score(y_train, y_h))
# Leaderboard training
print(automl.leaderboard())
automl.sprint_statistics()

# This works, with 0.6725 accuracy on test set as opposed to previous. With
# extended training time of 500 iterations, resulting in 0.675

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


RF Accuracy 0.64


/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(
Fitting to the training data: 100%|██████████| 300/300 [04:52<00:00,  1.02it/s, The total time budget for this task is 0:05:00]


AutoML Accuracy On Test 0.6725
AutoML Accuracy on Training 1.0
          rank  ensemble_weight               type      cost   duration
model_id                                                               
2            1             0.44      random_forest  0.306922  27.785364
4            2             0.46      random_forest  0.307756  24.112124
9            3             0.02  gradient_boosting  0.384487  18.510952
5            4             0.08  gradient_boosting  0.496247   8.410860


'auto-sklearn results:\n  Dataset name: f8e8e5dc-9471-11ee-8178-0242ac1c000c\n  Metric: accuracy\n  Best validation score: 0.693078\n  Number of target algorithm runs: 13\n  Number of successful target algorithm runs: 4\n  Number of crashed target algorithm runs: 2\n  Number of target algorithms that exceeded the time limit: 7\n  Number of target algorithms that exceeded the memory limit: 0\n'